In [1]:
import numpy as np
import matplotlib.pyplot as plt
from chainer import Chain, Variable,optimizers
import chainer.functions as F
import chainer.links as L
import random

/Users/linzexiang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### We need to append all the data and shuffle it. so it would scatter properly.

Using the cleaned data which store in advance.

In [2]:
f = open('Data1.txt', 'r')
x1 = f.readlines()
f.close()

In [3]:
f = open('Data2.txt', 'r')
x2 = f.readlines()
f.close()

In [4]:
f = open('Data3.txt', 'r')
x3 = f.readlines()
f.close()

In [5]:
totalx = x1+x2+x3
random.shuffle(totalx)
x = totalx

### X now become a list of string as we read from txt file.
we need to convert the string into list.

[1:29999] is all the input x

[30001:30003] is the score. or or [30001:30004] if it is 100(as all scores is between 80-100)

In [6]:
import time

data1_fin = []
start = time.time()
for index in range(10000):
    temp_data = x[index][1:29999]
    if x[index][30003] == '0': #the score is 100
        score = int(x[index][30001:30004])
    else: #the score is between 80-99
        score = int(x[index][30001:30003])
    indx = int(x[index][30005:-2])
    temp_data1 = temp_data.split(', ')
    data = [int(i) for i in temp_data1]
    data.append(score)
    data.append(indx)
    data1_fin.append(data)

end=time.time()
print(end-start)

23.319114923477173


## Setting up train and test data

In [7]:
x = list()
y = list()

for i in range(len(data1_fin)):
    x.append(data1_fin[i][0:-2])
    y.append([data1_fin[i][-2]])

In [8]:
number_of_data = len(x)
# variance_of_noise = 0.12
train_ratio = 0.8 #how much data will be used in training
slice_position = int(number_of_data * train_ratio)
x_train = x[:slice_position]
y_train = y[:slice_position]
x_train = Variable(np.array(x_train,dtype=np.float32))
y_train = Variable(np.array(y_train,dtype=np.float32))

x_test = x[slice_position:]
y_test = y[slice_position:]
x_test = Variable(np.array(x_test,dtype=np.float32))
y_test = Variable(np.array(y_test,dtype=np.float32))

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from chainer import Chain, Variable,optimizers
import chainer.functions as F
import chainer.links as L

start = time.time()
#### Defining a class for our NN model
class Regression(Chain):
    def __init__(self):
        super(Regression,self).__init__(
            ### One hidden layer
            l1 = L.Linear(10000,4000),
            l2 = L.Linear(4000,1),
    )

    def __call__(self,x):
        ### We define non-linear function here
        ### We use relu but I suggest to use sigmoid just because I like it!
        h = F.relu(self.l1(x))
        h = self.l2(h)
        return h

if __name__ == "__main__":
    #Parameters
    batchsize = 100
    max_epoch = 10
        
    model = Regression()
    optimizer = optimizers.MomentumSGD(lr=0.01,momentum=0.9)
    optimizer.setup(model)

    #Train
    N = len(x_train)
    perm = np.random.permutation(N)
    for epoch in range(max_epoch):
        for i in range(0,N,batchsize):
            x_train_batch = x_train[perm[i:i + batchsize]]
            y_train_batch = y_train[perm[i:i + batchsize]]

            ## Before calculating GD, we need to clear the gradients.
            model.cleargrads()
            loss = F.mean_squared_error(model(x_train_batch),y_train_batch)
            ### I beleive this one below calculate backward, which means calculating gradient for each parameter
            loss.backward()
            ### Now, using the gradient calculated above, they update the parameter
            optimizer.update()
            
        print("epoch:",epoch,"loss:",loss.data)

#Test
    model.cleargrads()
    t_test = model(x_test)
    loss = F.mean_squared_error(t_test,y_test)
    print("loss of test data:",loss.data)

    #x_grandtruth = np.arange(-1.7,1.7,0.1)
    #y_grandtruth = x_grandtruth**4-2*x_grandtruth**2+0.5*x_grandtruth+2.0

    #Plot1
    # plt.plot(x_grandtruth,y_grandtruth,color="red",label="Grand Truth")
    
end=time.time()
print("used time: ", end-start)

epoch: 0 loss: 12.778955
epoch: 1 loss: 9.97185
epoch: 2 loss: 9.988987
epoch: 3 loss: 9.989281
epoch: 4 loss: 9.675341
epoch: 5 loss: 4.1746864
epoch: 6 loss: 3.3703365
epoch: 7 loss: 3.1048756
epoch: 8 loss: 3.0345592
epoch: 9 loss: 3.290944
loss of test data: 4.169567
used time:  603.5054240226746
